In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_items = 32    # number of items
dim = 2           # dimension of constraints
caps = [20] * dim # capacity
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# generate data from PyEPO
import pyepo
weights, _, c = pyepo.data.knapsack.genData(num_data=num_data, num_features=5, num_items=num_items, dim=dim, deg=4, noise_width=0.5)
# data type
c = c.astype(np.float32)
# data split
from sklearn.model_selection import train_test_split
c_train, c_dev = train_test_split(c, test_size=1000, random_state=42, shuffle=True)
c_train, c_test = train_test_split(c_train, test_size=1000, random_state=42, shuffle=True)
# normalize
c_min = np.min(c_train, axis=0, keepdims=True)
c_max = np.max(c_train, axis=0, keepdims=True)
c_train_norm = (c_train - c_min) / (c_max - c_min)
c_min = np.min(c_dev, axis=0, keepdims=True)
c_max = np.max(c_dev, axis=0, keepdims=True)
c_dev_norm = (c_dev - c_min) / (c_max - c_min)
c_min = np.min(c_test, axis=0, keepdims=True)
c_max = np.max(c_test, axis=0, keepdims=True)
c_test_norm = (c_test - c_min) / (c_max - c_min)

Auto-Sklearn cannot be imported.


In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"c":c_train_norm}, name="train")
data_test = DictDataset({"c":c_test_norm}, name="test")
data_dev = DictDataset({"c":c_dev_norm,}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msKnapsack
model = msKnapsack(weights, caps, timelimit=60)

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for c in tqdm(c_test):
    # set params
    model.set_param_val({"c":c})
    # solve
    tick = time.time()
    xval, objval = model.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(c))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/kp_exact_32-2.csv")

## Heuristic - Round

In [36]:
from src.heuristic import floor_round

In [37]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for c in tqdm((c_test)):
    # set params
    model.set_param_val({"c":c})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = floor_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(c))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/kp_heur_rnd_32-2.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:13<00:00,  7.51it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     27.230000               0.0      0.127379
std      17.369421               0.0      0.015239
min       6.000000               0.0      0.113603
25%      15.000000               0.0      0.119143
50%      23.000000               0.0      0.120829
75%      34.000000               0.0      0.133908
max     131.000000               0.0      0.385531
Number of infeasible solution: 0


## Heuristic - RENS

In [20]:
from src.heuristic import rens

In [23]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for c in tqdm(c_test):
    # set params
    model.set_param_val({"c":c})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = rens(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(c))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/kp_heur_rens_32-2.csv")

 41%|████████████████████████████████▋                                              | 413/1000 [01:50<02:37,  3.73it/s]


KeyboardInterrupt: 

## Heuristic - N1

In [12]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for c in tqdm(c_test):
    # set params
    model.set_param_val({"c":c})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(c))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/kp_heur_n1_32-2.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:08<00:00,  7.76it/s]


       Obj Val  Constraints Viol  Elapsed Time
count   1000.0            1000.0   1000.000000
mean       4.0               0.0      0.127728
std        0.0               0.0      0.012178
min        4.0               0.0      0.108544
25%        4.0               0.0      0.123038
50%        4.0               0.0      0.124130
75%        4.0               0.0      0.125644
max        4.0               0.0      0.386401
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 1    # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmKnapsack
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_items, outsize=num_items, bias=True, linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["c"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=2*num_items, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_items)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["c"], var_keys=["x"],  output_keys=["x_rnd"], 
                       bin_ind={"x":range(num_items)}, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmKnapsack(["c", "x_rnd"], weights, caps, penalty_weight)
loss_fn2 = nmKnapsack(["c", "x"], weights, caps, penalty_weight, output_key="loss2")

In [10]:
from src.problem.neuromancer.trainer import trainer2
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer2(components, loss_fn, loss_fn2, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev, loss_weight=10)

Epoch 0, Validation Loss: 134.94
Epoch 1, Validation Loss: 22.04
Epoch 2, Validation Loss: 50.36
Epoch 3, Validation Loss: 30.92
Epoch 4, Validation Loss: 25.67
Epoch 5, Validation Loss: 19.62
Epoch 6, Validation Loss: 18.95
Epoch 7, Validation Loss: 23.02
Epoch 8, Validation Loss: 8.87
Epoch 9, Validation Loss: 7.26
Epoch 10, Validation Loss: 8.61
Epoch 11, Validation Loss: 7.62
Epoch 12, Validation Loss: 9.13
Epoch 13, Validation Loss: 14.34
Epoch 14, Validation Loss: 13.05
Epoch 15, Validation Loss: 9.72
Epoch 16, Validation Loss: 10.69
Epoch 17, Validation Loss: 8.96
Epoch 18, Validation Loss: 7.38
Epoch 19, Validation Loss: 8.46
Epoch 20, Validation Loss: 8.72
Epoch 21, Validation Loss: 11.12
Epoch 22, Validation Loss: 10.18
Epoch 23, Validation Loss: 4.85
Epoch 24, Validation Loss: 7.35
Epoch 25, Validation Loss: 7.87
Epoch 26, Validation Loss: 5.73
Epoch 27, Validation Loss: 8.89
Epoch 28, Validation Loss: 9.30
Epoch 29, Validation Loss: 6.47
Epoch 30, Validation Loss: 8.25
Epoc

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for c, c_orig in tqdm(list(zip(c_test_norm, c_test))):
    # data point as tensor
    datapoints = {"c": torch.tensor(np.array([c]), dtype=torch.float32).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"c":c_orig})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_items):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(c))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/kp_lr_32-2.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 187.22it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     14.506000               0.0      0.002267
std       6.868125               0.0      0.000667
min       5.000000               0.0      0.000998
25%      11.000000               0.0      0.002000
50%      13.000000               0.0      0.002001
75%      17.000000               0.0      0.002512
max      99.000000               0.0      0.008106
Number of infeasible solution: 0


## Learnable Threshold

In [48]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [49]:
# hyperparameters
penalty_weight = 1    # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [50]:
# set problem
import neuromancer as nm
from src.problem import nmKnapsack
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_items, outsize=num_items, bias=True, linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["c"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=2*num_items, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_items)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["c"], var_keys=["x"],  output_keys=["x_rnd"], 
                          bin_ind={"x":range(num_items)}, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmKnapsack(["c", "x_rnd"], weights, caps, penalty_weight)

In [51]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 702.18
Epoch 1, Validation Loss: 0.00
Epoch 2, Validation Loss: 0.00
Epoch 3, Validation Loss: -0.00
Epoch 4, Validation Loss: -0.04
Epoch 5, Validation Loss: -0.38
Epoch 6, Validation Loss: -0.34
Epoch 7, Validation Loss: -0.57
Epoch 8, Validation Loss: -0.63
Epoch 9, Validation Loss: -0.89
Epoch 10, Validation Loss: -0.89
Epoch 11, Validation Loss: -0.77
Epoch 12, Validation Loss: -0.77
Epoch 13, Validation Loss: -0.85
Epoch 14, Validation Loss: -0.77
Epoch 15, Validation Loss: -0.64
Epoch 16, Validation Loss: -0.51
Epoch 17, Validation Loss: -0.59
Epoch 18, Validation Loss: -0.59
Epoch 19, Validation Loss: -0.55
Epoch 20, Validation Loss: -0.43
Epoch 21, Validation Loss: -0.59
Epoch 22, Validation Loss: -0.51
Epoch 23, Validation Loss: -0.53
Epoch 24, Validation Loss: -0.54
Epoch 25, Validation Loss: -0.53
Epoch 26, Validation Loss: -0.59
Epoch 27, Validation Loss: -0.72
Epoch 28, Validation Loss: -0.64
Epoch 29, Validation Loss: -0.68
Epoch 30, Validation 

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for c, c_orig in tqdm(list(zip(c_test_norm, c_test))):
    # data point as tensor
    datapoints = {"c": torch.tensor(np.array([c]), dtype=torch.float32).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"c":c_orig})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_items):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(c))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/kp_lt_32-2.csv")